# Rolling Planning

In [2]:
import re
import json
import random
import numpy as np
import pandas as pd
import pulp
import random

In [3]:
import utils.checker as check
import utils.presenter as show
import utils.schedule_interarrival as sit
import utils.schedule_solver__with_arrivals as ssv

In [4]:
pd.set_option('display.max_rows', 30)

In [5]:
# Datei laden
with open("data/jobshop_instances.json", "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
show.print_jobs(instance)

job 0:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
job 1:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
job 2:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
job 3:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
job 4:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
job 5:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
job 6:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
job 7:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
job 8:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
job 9:  [[1, 85], [0, 13], [2, 61], [6, 7], [8, 64], [9, 76], [5, 47], [3, 52], [4, 90], [7, 45]]



In [6]:
### ------

In [7]:

def create_new_jobs(job_set_dict: dict, new_instance: dict, shuffle: bool = False, seed: int = 50):
    """
    Erzeugt neue Jobs mit fortlaufenden IDs. 
    Der Offset ist die nächsthöhere Nummer nach der größten vorhandenen Job-ID,
    nicht-matching IDs bekommen einen Suffix von 0.
    """
    # 1) Items aus dem new_instance-Dict auslesen
    items = list(new_instance.items())

    # 2) Optional mischen
    if shuffle:
        random.seed(seed)
        random.shuffle(items)

    # 3) Offset ermitteln: höchste vorhandene Nummer + 1,
    #    nicht-matching Job-Keys zählen als 0
    suffixes = []
    for job_id in (job_set_dict or {}):
        m = re.search(r"Job_(\d+)$", job_id)
        suffixes.append(int(m.group(1)) if m else 0)

    offset = (max(suffixes) + 1) if suffixes else 0

    # 4) Neues Dict zusammenbauen
    new_jobs = {}
    for i, (_, ops) in enumerate(items):
        job_name = f"Job_{offset + i:03d}"
        new_jobs[job_name] = ops

    return new_jobs


def create_new_arrivals(numb_jobs: int, mean_interarrival_time: float, start_time: int = 0, random_seed_times: int = 122):
    # 1) Seed setzen für Reproduzierbarkeit
    np.random.seed(random_seed_times)

    # 2) Interarrival-Zeiten erzeugen
    interarrival_times = np.random.exponential(scale=mean_interarrival_time, size=numb_jobs)

    # 3) Kumulieren ab last_arrival und auf 2 Nachkommastellen runden
    new_arrivals = np.round(start_time + np.cumsum(interarrival_times), 2)

    return new_arrivals


In [27]:
def create_new_jobs_with_arrivals_for_one_day(old_job_dict: dict, old_arrivals_df: pd.DataFrame, new_job_dict: dict, 
                                              u_b_mmax: float = 0.9, shuffle: bool = False):

    new_day_start = 0
    
    if old_arrivals_df is not None and not old_arrivals_df.empty:
        last_old_arrival = old_arrivals_df["Arrival"].max()
        new_day_start = ((last_old_arrival // 1440) + 1) * 1440

        
    # 1) Instanz vervielfachen
    new_jssp_data = {}
    job_dict_temp = old_job_dict.copy() if old_job_dict else {}

    
    for i in range(3):
        
        # abwechselnd shufflen oder nicht
        shuffle_flag = shuffle if (i % 2 == 0) else not shuffle
        
        # erzeugt ein dict neuer Jobs
        new_jobs_i = create_new_jobs(job_dict_temp, new_job_dict, shuffle=shuffle_flag, seed=50)
        job_dict_temp  = new_jobs_i
        
        new_jssp_data.update(new_jobs_i)

    # 2) mittlere Zwischenankunftszeit berechnen
    mean_interarrival_time = sit.calculate_mean_interarrival_time(new_jssp_data, u_b_mmax)

    # 3) Ankunftszeiten generieren
    new_arrival_times = create_new_arrivals(len(new_jssp_data), mean_interarrival_time, new_day_start, random_seed_times=122)
    df_arrivals = pd.DataFrame({
        "Job": list(new_jssp_data.keys()),
        "Arrival": new_arrival_times
    })

    # Filtern
    df_arrivals = df_arrivals[
        (df_arrivals["Arrival"] >= new_day_start) &
        (df_arrivals["Arrival"] < new_day_start + 1440)
    ]

    # 5) new_jssp_data auf die tatsächlich verbleibenden Jobs kürzen
    valid_job_ids = set(df_arrivals["Job"])
    new_jssp_data = {job_id: ops for job_id, ops in new_jssp_data.items() if job_id in valid_job_ids}

    return new_jssp_data, df_arrivals


In [29]:
jssp_data, df_arrivals = create_new_jobs_with_arrivals_for_one_day(None, None, instance)
show.print_jobs(jssp_data)
df_arrivals

Job_000:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
Job_001:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
Job_002:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
Job_003:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
Job_004:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
Job_005:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
Job_006:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
Job_007:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
Job_008:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
Job_009:  [[1, 85], [0, 13], [2, 61], [6, 7], [8, 64], [9, 76], [5, 47], [3, 52], [4, 90], [7, 4

,Job,Arrival
0,Job_000,11.97
1,Job_001,96.90
2,Job_002,118.15
3,Job_003,160.29
4,Job_004,197.05
5,Job_005,263.34
6,Job_006,324.68
7,Job_007,341.61
8,Job_008,356.27
9,Job_009,441.18


In [10]:
jssp_data2, df_arrivals2 = create_new_jobs_with_arrivals_for_one_day(jssp_data, df_arrivals, instance)
show.print_jobs(jssp_data2)
df_arrivals2

Last old arrival: 1338.3
New day start: 1440.0
Job_027:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
Job_028:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
Job_029:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
Job_030:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
Job_031:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
Job_032:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
Job_033:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
Job_034:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
Job_035:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
Job_036:  [[1, 85], [0, 13], [2, 61], [6, 7], [8,

,Job,Arrival
0,Job_027,1451.97
1,Job_028,1536.90
2,Job_029,1558.15
3,Job_030,1600.29
4,Job_031,1637.05
5,Job_032,1703.34
6,Job_033,1764.68
7,Job_034,1781.61
8,Job_035,1796.27
9,Job_036,1881.18


In [11]:
jssp_data3, df_arrivals3 = create_new_jobs_with_arrivals_for_one_day(jssp_data2, df_arrivals2, instance)
show.print_jobs(jssp_data3)
df_arrivals3

Last old arrival: 2778.3
New day start: 2880.0
Job_054:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
Job_055:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
Job_056:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
Job_057:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
Job_058:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
Job_059:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
Job_060:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
Job_061:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
Job_062:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
Job_063:  [[1, 85], [0, 13], [2, 61], [6, 7], [8,

,Job,Arrival
0,Job_054,2891.97
1,Job_055,2976.90
2,Job_056,2998.15
3,Job_057,3040.29
4,Job_058,3077.05
5,Job_059,3143.34
6,Job_060,3204.68
7,Job_061,3221.61
8,Job_062,3236.27
9,Job_063,3321.18


In [23]:
import pandas as pd

def init_jobs_with_arrivals(new_job_dict: dict, day_num: int, mu_b_mmax: float = 0.9) -> (dict, pd.DataFrame):

    # Startwerte: keine alten Jobs oder Ankünfte
    old_job_dict = {}
    old_arrivals_df = pd.DataFrame(columns=["Job", "Arrival"])

    combined_jobs = {}
    arrivals_list = []

    # Für jeden Tag neu erzeugen und anhängen
    for _ in range(day_num):
        new_jobs, df_arrivals = create_new_jobs_with_arrivals_for_one_day(old_job_dict, old_arrivals_df, 
                                                                          new_job_dict, u_b_mmax=u_b_mmax)
        # Jobs sammeln
        combined_jobs.update(new_jobs)
        # Ankünfte sammeln
        arrivals_list.append(df_arrivals)

        # Für den nächsten Tag als Basis übernehmen
        old_job_dict = new_jobs
        old_arrivals_df = df_arrivals

    # Alle Ankünfte zu einem DataFrame zusammenführen
    df_all_arrivals = pd.concat(arrivals_list, ignore_index=True)

    return combined_jobs, df_all_arrivals


In [25]:
jssp_data_days, df_arrivals_days = init_jobs_with_arrivals(instance, 3)
show.print_jobs(jssp_data_days)
df_arrivals_days

New day start: 0
Last old arrival: 1338.3
New day start: 1440.0
Last old arrival: 2778.3
New day start: 2880.0
Job_000:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
Job_001:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
Job_002:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
Job_003:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
Job_004:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
Job_005:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
Job_006:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
Job_007:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
Job_008:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4,

,Job,Arrival
0,Job_000,11.97
1,Job_001,96.90
2,Job_002,118.15
3,Job_003,160.29
4,Job_004,197.05
...,...,...
76,Job_076,3988.85
77,Job_077,4157.26
78,Job_078,4185.31
79,Job_079,4199.70


In [31]:
def update_new_day(existing_jobs: dict, existing_arrivals_df: pd.DataFrame, new_job_dict: dict, u_b_mmax: float = 0.9, shuffle: bool = False):

    # 1) Neue Jobs + Ankünfte für einen Tag erzeugen
    new_jobs, df_arrivals_new = create_new_jobs_with_arrivals_for_one_day(existing_jobs, existing_arrivals_df, new_job_dict, u_b_mmax=u_b_mmax)
    
    # 2) Bestehende Jobs um die Neuen erweitern
    existing_jobs.update(new_jobs)

    # 3) Bestehende Ankünfte an die Neuen anhängen
    updated_arrivals_df = pd.concat([existing_arrivals_df, df_arrivals_new], ignore_index=True)

    return existing_jobs, updated_arrivals_df

In [33]:
jssp_data_days, df_arrivals_days = update_new_day(jssp_data_days, df_arrivals_days, instance)
show.print_jobs(jssp_data_days)
df_arrivals_days

Job_000:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
Job_001:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
Job_002:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
Job_003:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
Job_004:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
Job_005:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
Job_006:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
Job_007:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
Job_008:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
Job_009:  [[1, 85], [0, 13], [2, 61], [6, 7], [8, 64], [9, 76], [5, 47], [3, 52], [4, 90], [7, 4

,Job,Arrival
0,Job_000,11.97
1,Job_001,96.90
2,Job_002,118.15
3,Job_003,160.29
4,Job_004,197.05
...,...,...
103,Job_103,5428.85
104,Job_104,5597.26
105,Job_105,5625.31
106,Job_106,5639.70
